# WIP: Back-test

This example is still WIP.
* Load Data
* Back Test - the Basic Idea

In [1]:
from grandma_stock_valuation import FileLogger, loadPacakgeData
from grandma_stock_valuation import batchValuation, addCashPortfolio, getCorrelationWeight, allocatePortfolio

# Refer to example_0_FileLogger.ipynb for details of the FileLogger.
logger = FileLogger()
logPrint = logger.logPandas

### Load Data

For this example, I will use the stored package data.

To query data from Yahoo, please refer to *example_1_yahoo_data_loader.ipynb*.

In [2]:
d_instrument_data, d_instrument = loadPacakgeData(verbose=2)

d_instrument

VPL data contains 4273 rows, 4273 dates from 2005-03-10 to 2022-02-28.
IVV data contains 5479 rows, 5479 dates from 2000-05-19 to 2022-02-28.
EEMA data contains 2530 rows, 2530 dates from 2012-02-09 to 2022-02-28.
IEV data contains 5431 rows, 5431 dates from 2000-07-28 to 2022-02-28.


{'IVV': 'SP500',
 'VPL': 'Developed Asia-Pacific',
 'IEV': 'Europe',
 'EEMA': 'Emerging Asia'}

In [3]:
logPrint("Keys of d_instrument_data:", str(d_instrument_data.keys()))

logPrint("IVV (SP500 ETF):", d_instrument_data['IVV'].head())

2022-03-11 22:17:06,570 INFO Keys of d_instrument_data: dict_keys(['VPL', 'IVV', 'EEMA', 'IEV'])
2022-03-11 22:17:06,583 INFO IVV (SP500 ETF): 
        date       open       high        low      close  close_adj   volume
0 2000-05-19  142.65625  142.65625  140.25000  140.68750  94.121216   775500
1 2000-05-22  140.59375  140.59375  136.81250  139.81250  93.535789  1850600
2 2000-05-23  140.21875  140.21875  137.68750  137.68750  92.114151   373900
3 2000-05-24  137.75000  140.06250  136.65625  139.75000  93.494003   400300
4 2000-05-25  140.03125  140.93750  137.87500  138.46875  92.636810    69600



### Back Test - the Basic Idea

WIP
* back test is to understand how your strategy behaves under various scenanios, not to prove your strategy can beat xxx.
* most investment products are not built to beat xxx.
* historical performance cannot represent future performance.


### Back Test - WIP

In [4]:
from grandma_stock_valuation.back_test import GrandmaBackTester

backtester = GrandmaBackTester(
    backtest_years=10,
    adjust_freq_months=3,
    init_parameters={'recent_months':0, 'train_years':10, 'min_train_years':5, 'date_end':None},
    fit_parameters={'price_col':'close_adj', 'log':True, 'n_std':1.5},
    valuate_parameters={'min_annual_return':0.01},
    allocation_parameters={'transformation':'sigmoid', 'scale':1},
    with_cash=True,
    with_correlation_weights=True,
    verbose=1,
    printfunc=logPrint
)

backtester.runBackTest(d_instrument_data)

backtester.df_average_value


2022-03-11 22:17:06,738 INFO To backtest 10 years, from 2012-03-01 to 2022-02-28
2022-03-11 22:17:06,739 INFO EEMA's start date 2012-02-09 is beyond 2007-03-01 for full back-test.
2022-03-11 22:17:07,263 INFO Adjust portfolio on 2012-06-01, total value = 0.909008
2022-03-11 22:17:07,681 INFO Adjust portfolio on 2012-09-04, total value = 0.998287
2022-03-11 22:17:08,099 INFO Adjust portfolio on 2012-12-03, total value = 1.036429
2022-03-11 22:17:08,591 INFO Adjust portfolio on 2013-03-01, total value = 1.067009
2022-03-11 22:17:09,011 INFO Adjust portfolio on 2013-06-03, total value = 1.096861
2022-03-11 22:17:09,410 INFO Adjust portfolio on 2013-09-03, total value = 1.098930
2022-03-11 22:17:09,826 INFO Adjust portfolio on 2013-12-02, total value = 1.108694
2022-03-11 22:17:10,215 INFO Adjust portfolio on 2014-03-03, total value = 1.108981
2022-03-11 22:17:10,648 INFO Adjust portfolio on 2014-06-02, total value = 1.109287
2022-03-11 22:17:11,089 INFO Adjust portfolio on 2014-09-02, tot

,ticker,avg_pct_allocation
0,EEMA,0.127810
1,IEV,0.189098
2,IVV,0.195924
3,VPL,0.081611
4,cash,0.405557


In [5]:
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [6]:
def _add_date_to_portfolio(item):
    dt = item[0]
    df = item[1].copy()
    df['date'] = dt
    df = df[['date','ticker','current_portfolio_pct']]
    return df

df_daily_portfolio_pct = pd.concat(map(_add_date_to_portfolio, backtester.d_portfolio.items()), ignore_index=True).fillna(0)

# In the chart, Grandma's allocation be displayed as the cash %
index_cash = df_daily_portfolio_pct['ticker']=='cash'
df_daily_portfolio_pct.loc[index_cash, 'ticker'] = 'Grandma'

df_daily_portfolio_pct.head(6)

,date,ticker,current_portfolio_pct
0,2012-03-01,VPL,0.000000
1,2012-03-01,IVV,0.133163
2,2012-03-01,EEMA,0.000000
3,2012-03-01,IEV,0.485075
4,2012-03-01,Grandma,0.381761
5,2012-03-02,VPL,0.000000


In [15]:
df_grandma_growth = pd.DataFrame({'date':backtester.d_total_value.keys(), 'Grandma':backtester.d_total_value.values()})
df_grandma_growth = df_grandma_growth.melt(id_vars='date', var_name='ticker', value_name='growth')
df_grandma_growth = df_grandma_growth.merge(df_daily_portfolio_pct, 'left', ['date','ticker']).fillna(0)
df_grandma_growth.rename(columns={'current_portfolio_pct':'hover_text'}, inplace=True)
df_grandma_growth['hover_text'] = (100*df_grandma_growth['hover_text']).round(1).astype(str) + '% Cash'

df_instrument_growth = backtester._df_instrument_prices.iloc[backtester._index_start:].copy()
d_first_prices = df_instrument_growth.apply(lambda se: se.dropna().iloc[0])

df_instrument_growth = df_instrument_growth.apply(lambda se: se/d_first_prices[se.name])
df_instrument_growth.columns = [c.replace('price_', '') for c in df_instrument_growth.columns]
df_instrument_growth = df_instrument_growth.reset_index()
df_instrument_growth = df_instrument_growth.melt(id_vars='date', var_name='ticker', value_name='growth')

df_instrument_growth = df_instrument_growth.merge(df_daily_portfolio_pct, 'left', ['date','ticker']).fillna(0)
df_instrument_growth.rename(columns={'current_portfolio_pct':'hover_text'}, inplace=True)
df_instrument_growth['hover_text'] = (100*df_instrument_growth['hover_text']).round(1).astype(str) + '% Portfolio'

df_instrument_growth = pd.concat([df_grandma_growth, df_instrument_growth], ignore_index=True) # sequence matters here

df_instrument_growth.head(3)

,date,ticker,growth,hover_text
0,2012-03-01,Grandma,1.000000,38.2% Cash
1,2012-03-02,Grandma,0.994148,38.4% Cash
2,2012-03-05,Grandma,0.991679,38.5% Cash


In [16]:
fig = px.line(df_instrument_growth, x='date', y='growth',
              color='ticker', color_discrete_map={'Grandma':'white'},
              custom_data = ['hover_text'],
              )

fig.update_traces(hovertemplate='%{y:.2f}<br>%{customdata}')
fig.update_traces(line=dict(width=0.75))
fig['data'][0]['line']['width'] = 1.5

fig.update_layout(template='plotly_dark', title='Back-test',
                  xaxis_title='date', yaxis_title='growth',
                  hovermode='x unified',
                  xaxis_hoverformat='%Y-%m-%d')

index_adj_scatter = df_grandma_growth['date'].isin(backtester.d_adjustments.keys())
df_adj_scatter = df_grandma_growth[index_adj_scatter][['date','growth']]

fig.add_trace(go.Scatter(x=df_adj_scatter['date'], y=df_adj_scatter['growth'], mode='markers', showlegend=False,
                         marker_symbol='diamond-tall', marker_color="lightskyblue", marker_size=8,
                         hoverinfo='skip'))

fig.show()